# Pilot 2 -- Compute the univariate activations of various SR stim-context integrations
### Ex: Color stims ([red, red] v. [red, blue] v. [blue, red] v. [blue, blue])

## Use SVM classifications to input stimuli (both modalities: visual and auditory)
## Using Ciric-style postprocessing

## Takuya Ito
#### 12/06/2018

In [16]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns
import h5py
os.sys.path.append('glmScripts/')
import taskGLMPipeline as tgp

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [17]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [18]:
def loadInputActivity(subj,inputtype):
    x = tgp.loadTaskTiming('013','ALL')
    stimIndex = np.asarray(x['stimIndex'])
    ind = np.where(stimIndex==inputtype)[0]+1
    
    datadir = basedir + 'data/postProcessing/hcpPostProcCiric/'
    h5f = h5py.File(datadir + subj + '_glmOutput_data.h5','r')
    data = h5f['taskRegression/ALL_24pXaCompCorXVolterra_taskReg_betas_canonical'][:].copy()
    data = data[:,ind].copy()
    h5f.close()
    return data


# 1.0 Run across subject univariate analysis on RED SR rules
### 4-way decoding
* both red
* notboth red
* either red
* neither red

## 1.1 Load data for RED SR integration

In [19]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srRed'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

## 1.2 Perform univariate analyses

In [20]:
# Perform

group_stats = np.zeros((data_task.shape[0],data_task.shape[1])) #t, p, t_thresh
for stim in range(data_task.shape[1]):
    t, p = stats.ttest_1samp(data_task[:,stim,:],0,axis=1)
    h0, qs = mc.fdrcorrection0(p)
    group_stats[:,stim] = np.multiply(t,h0)
    
    

## 1.3 Map activations back to cortical surface

In [21]:
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Univariate'
np.savetxt(outdir + filename + '.csv', group_stats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

# 2.0 Run across subject univariate on VERTICAL SR rules
### 4-way decoding
* both vertical
* notboth vertical
* either vertical
* neither vertical

## 1.1 Load data for VERTICAL SR integration

In [22]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srVertical'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

## 1.2 Perform univariate analyses

In [23]:
# Perform

group_stats = np.zeros((data_task.shape[0],data_task.shape[1])) #t, p, t_thresh
for stim in range(data_task.shape[1]):
    t, p = stats.ttest_1samp(data_task[:,stim,:],0,axis=1)
    h0, qs = mc.fdrcorrection0(p)
    group_stats[:,stim] = np.multiply(t,h0)
    
    

## 1.3 Map activations back to cortical surface

In [24]:
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Univariate'
np.savetxt(outdir + filename + '.csv', group_stats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

# 3.0 Run across subject univariate analysis on HIGH SR rules
### 4-way decoding
* both high
* notboth high
* either high
* neither high

## 1.1 Load data for HIGH SR integration

In [25]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srHigh'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

## 1.2 Perform univariate analyses

In [26]:
# Perform

group_stats = np.zeros((data_task.shape[0],data_task.shape[1])) #t, p, t_thresh
for stim in range(data_task.shape[1]):
    t, p = stats.ttest_1samp(data_task[:,stim,:],0,axis=1)
    h0, qs = mc.fdrcorrection0(p)
    group_stats[:,stim] = np.multiply(t,h0)
    
    

/usr/local/anaconda2/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/anaconda2/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/anaconda2/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/usr/local/anaconda2/lib/python2.7/site-packages/statsmodels/stats/multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha


## 1.3 Map activations back to cortical surface

In [27]:
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Univariate'
np.savetxt(outdir + filename + '.csv', group_stats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

# 4.0 Run across subject univariate analysis on CONSTANT SR rules
### 4-way decoding
* both constant
* notboth constant
* either constant
* neither constant

## 1.1 Load data for CONSTANT SR integration

In [28]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srConstant'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

## 1.2 Perform univariate analyses

In [29]:
# Perform

group_stats = np.zeros((data_task.shape[0],data_task.shape[1])) #t, p, t_thresh
for stim in range(data_task.shape[1]):
    t, p = stats.ttest_1samp(data_task[:,stim,:],0,axis=1)
    h0, qs = mc.fdrcorrection0(p)
    group_stats[:,stim] = np.multiply(t,h0)
    
    

## 1.3 Map activations back to cortical surface

In [30]:
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Univariate'
np.savetxt(outdir + filename + '.csv', group_stats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0